In [21]:
import kaggle

In [22]:
! kaggle datasets init -p /app/_data/models/final/Eff5HandComplex/

Data package template written to: /app/_data/models/final/Eff5HandComplex/dataset-metadata.json


In [23]:
! kaggle datasets create -p /app/_data/models/final/Eff5HandComplex/

Starting upload for file Eff5HandComplex_5.h5
100%|█████████████████████████████████████████| 327M/327M [08:27<00:00, 676kB/s]
Upload successful: Eff5HandComplex_5.h5 (327MB)
Starting upload for file Eff5HandComplex_3.h5
100%|█████████████████████████████████████████| 327M/327M [08:30<00:00, 672kB/s]
Upload successful: Eff5HandComplex_3.h5 (327MB)
Skipping folder: backup; use '--dir-mode' to upload folders
Starting upload for file Eff5HandComplex_2.h5
100%|█████████████████████████████████████████| 327M/327M [08:53<00:00, 643kB/s]
Upload successful: Eff5HandComplex_2.h5 (327MB)
Starting upload for file Eff5HandComplex_1.h5
100%|█████████████████████████████████████████| 327M/327M [08:26<00:00, 677kB/s]
Upload successful: Eff5HandComplex_1.h5 (327MB)
Starting upload for file Eff5HandComplex_4.h5
100%|█████████████████████████████████████████| 327M/327M [08:38<00:00, 661kB/s]
Upload successful: Eff5HandComplex_4.h5 (327MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload f

In [1]:
! pip install albumentations
import albumentations

     |████████████████████████████████| 72 kB 438 kB/s eta 0:00:011
     |████████████████████████████████| 30.2 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 51.0 MB 91 kB/s s eta 0:00:01
     |████████████████████████████████| 3.3 MB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 23.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 165 kB 25.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

%matplotlib inline

In [3]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
PATH = "/app/_data/"
BATCH_SIZE = 26
SEED = 37
IMAGE_SIZE = 456
NUM_CLASSES = 6
TRAIN_IMG_PATH = "/app/_data/456_npy/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = [
    "complex",
    "frog_eye_leaf_spot",
    "healthy",
    "powdery_mildew",
    "rust",
    "scab",
]
wrong = [
    "ead085dfac287263.jpg",
    "95276ccd226ad933.jpg",
    "da8770e819d2696d.jpg",
    "cd3a1d64e6806eb5.jpg",
    "ccec54723ff91860.jpg",
]

In [5]:
df_labels = pd.read_csv("/app/_data/df_csv/labels_21_20.csv", index_col=[0])
df_labels = df_labels.query("image not in @wrong").reset_index(drop=True)

In [6]:
complex_labeled1 = pd.read_csv('/app/_data/models/final/Eff5HandComplex/backup/complex1_labels.csv')
complex_labeled2 = pd.read_csv('/app/_data/models/final/Eff5HandComplex/backup/complex2_labels.csv')
complex_labeled = pd.concat([complex_labeled1, complex_labeled2], axis=0, ignore_index=True)
complex_labeled['image'] = complex_labeled['image'].apply(lambda x: x.split('/')[-1])
complex_labeled['image'] = complex_labeled['image'].apply(lambda x: x.split('_')[0])
complex_labeled['image'] = complex_labeled['image'].apply(lambda x: x.split('.')[0])
complex_labeled['image'] = complex_labeled['image']+'.jpg'
complex_labeled = complex_labeled[['image', 'choice']]
complex_labeled.columns = ['image', 'labels']
complex_labeled = complex_labeled.join(complex_labeled['labels'].str.get_dummies(' '))

In [7]:
df= pd.DataFrame(columns = complex_labeled.columns)
for image in complex_labeled[complex_labeled['image'].duplicated(keep=False)]['image']:
    df.loc[image, 'image'] = image
    for column in feature_columns:
        if complex_labeled[complex_labeled['image']==image][column].sum() == 2:
            df.loc[image, column] =1
        else:
            df.loc[image, column] = 0
    df.loc[image, 'labels'] = ' '.join([feature_columns[i] for i in range(len(feature_columns)) if df.loc[image, feature_columns].values[i]==1])

In [8]:
dup_list= complex_labeled[complex_labeled['image'].duplicated(keep=False)]['image'].tolist()
complex_labeled = complex_labeled.query('image not in @dup_list')
complex_labeled = pd.concat([complex_labeled, df], axis=0, ignore_index=True)

In [9]:
complex_labeled = complex_labeled.dropna(axis=0)

In [10]:
# complex_labeled.to_csv('/app/_data/models/final/Eff5HandComplex/backup/complex_labels.csv', index=False)

In [11]:
df_complex = df_labels.query('labels == "complex"').reset_index(drop=True)
df_without_complex = df_labels.query('labels != "complex"').reset_index(drop=True)
df_labels = pd.concat([df_without_complex, complex_labeled], axis=0, ignore_index=True).sample(frac=1, random_state=SEED).reset_index(drop=True)
df_labels["image"] = df_labels["image"].str.replace(".jpg", ".npy")

In [12]:
df_labels

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,f49ab56b4f920b21.npy,scab,0,0,0,0,0,1
1,e924dd52942f9e4c.npy,rust,0,0,0,0,1,0
2,fa3f80ea9c4b1a58.npy,rust,0,0,0,0,1,0
3,adc27f0e7ca55510.npy,scab,0,0,0,0,0,1
4,db652835372c9758.npy,rust,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
20299,dbd2388b673c44e4.npy,healthy,0,0,1,0,0,0
20300,a4b57ed29e341619.npy,frog_eye_leaf_spot,0,1,0,0,0,0
20301,87b58bdb956a8164.npy,scab,0,0,0,0,0,1
20302,c4ef976cb1ca304a.npy,frog_eye_leaf_spot,0,1,0,0,0,0


In [13]:
# for 380*380
transform = albumentations.Compose(
    [
        albumentations.CLAHE(p=0.1, clip_limit=(1, 2), tile_grid_size=(8, 8)),
        albumentations.OneOf(
            [
                albumentations.MotionBlur((3, 3)),
                albumentations.MedianBlur(blur_limit=3),
                albumentations.GaussianBlur(blur_limit=(3, 3), sigma_limit=0),
                albumentations.Blur(blur_limit=(3, 3)),
            ],
            p=0.2,
        ),
        albumentations.OneOf(
            [
                albumentations.GaussNoise(var_limit=[10, 50], mean=1),
                albumentations.ISONoise(intensity=(0.1, 1), color_shift=(0.01, 0.05)),
                albumentations.ImageCompression(
                    quality_lower=70, quality_upper=100, compression_type=1
                ),
                albumentations.MultiplicativeNoise(
                    multiplier=(0.95, 1.05), per_channel=True, elementwise=True
                ),
                albumentations.Downscale(
                    scale_min=0.6, scale_max=0.99, interpolation=4
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=(-7, 7),
                    sat_shift_limit=(-10, 10),
                    val_shift_limit=(-10, 10),
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=0.15,
                    contrast_limit=0.2,
                    brightness_by_max=True,
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.OpticalDistortion(
                    distort_limit=0.05,
                    shift_limit=0.05,
                    border_mode=2,
                ),
                albumentations.ElasticTransform(
                    alpha=2.0,
                    sigma=50.0,
                    alpha_affine=10.0,
                    interpolation=0,
                    border_mode=2,
                ),
                albumentations.GridDistortion(
                    num_steps=5, distort_limit=0.3, interpolation=0, border_mode=2
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HorizontalFlip(),
                albumentations.VerticalFlip(),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.Rotate(
                    limit=(-180, 180), interpolation=0, border_mode=2
                ),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=180,
                    interpolation=0,
                    border_mode=2,
                ),
            ],
            p=0.5,
        ),
    ]
)

In [14]:
# plt.figure(figsize=(20,20))
# for i, img in enumerate(os.listdir(TRAIN_IMG_PATH)[100:120]):
#     img = np.load(TRAIN_IMG_PATH+img)
#     plt.subplot(4,5,i+1)
#     plt.imshow(img)
# plt.show();

In [15]:
class Generator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        images_src_dir,
        batch_size,
        target_image_size,
        shuffle=False,
        augment=True,
        crop=False,
        resize=False,
        normalize=False,
    ):
        self.shuffle = shuffle
        self.batch_size = BATCH_SIZE
        self.df = df
        self.images_dir = images_src_dir
        self.target_image_size = (IMAGE_SIZE, IMAGE_SIZE)
        self.augment = augment
        self.crop = crop
        self.resize = resize
        self.normalize = normalize
        # create label index map
        self.labels = self._read_labels()
        self.n_samples = self.df.shape[0]
        self.n_batches = self.n_samples // self.batch_size
        # shuffle data, also repeated after each epoch if needed
        if self.shuffle:
            np.random.shuffle(self.labels)

    def _read_labels(self):
        """
        Returns list images mapping to 1-hot label
        """

        # label indexes
        label_ixs = self.df[feature_columns].values
        image_ixs = self.df["image"].values
        labels = []

        for i in range(len(image_ixs)):
            labels.append([image_ixs[i], label_ixs[i]])
        return labels

    def __len__(self):
        """
        Length in batches
        """
        return self.n_batches

    def __getitem__(self, b_ix):
        """
        Produce batch, by batch index
        """

        assert b_ix < self.n_batches

        b_X = np.zeros(
            (self.batch_size, self.target_image_size[0], self.target_image_size[1], 3),
            dtype=np.uint8,
        )

        b_Y = np.zeros(
            (self.batch_size, self.df[feature_columns].shape[1]),
            dtype=np.uint8,
        )

        for i in range(self.batch_size):
            b_X[i], b_Y[i] = self.get_one(
                i + self.batch_size * b_ix,
            )

        return (b_X, b_Y)

    def get_one(self, one_ix):
        """
        Get single item by absolute index
        """
        id = self.labels[one_ix][0]
        src_file = self.images_dir + id

        # read file
        x = np.load(src_file)
        if self.crop:
            coord = self.df[self.df["image"] == id][
                ["x_min", "y_min", "x_max", "y_max"]
            ].values[0]
            orig_hight = x.shape[0]
            orig_width = x.shape[1]
            x_min = coord[0]
            y_min = coord[1]
            x_max = coord[2]
            y_max = coord[3]
            x = x[
                np.int(y_min * orig_hight) : np.int(y_max * orig_hight),
                np.int(x_min * orig_width) : np.int(x_max * orig_width),
            ]

        y = self.labels[one_ix][1]

        # augment
        if self.augment:
            x = self._augment_image(x)

        # normalize (sample-wise)
        if self.normalize:
            x = x.astype(np.float32)
            x = x - np.mean(x, axis=(0, 1))
            x = x / np.std(x, axis=(0, 1))
        return x.astype(np.uint8), y

    def _augment_image(self, x):
        """
        Randomply augment image
        """

        x = transform(image=x)["image"]
        return x

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.labels)

In [16]:
gen = Generator(
    df=df_labels.sample(100),
    images_src_dir=TRAIN_IMG_PATH,
    target_image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    augment=False,
    crop=False,
    resize=False,
)

In [17]:
gen.__len__()

3

In [18]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [19]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB5(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b5_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [20]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_labels["image"], df_labels["labels"])
):
    train, valid = df_labels.loc[train_index], df_labels.loc[valid_index]
    model_name = "Eff5HandComplex_" + str(i + 1) + ".h5"
    log_dir = "logs_Eff5HandComplex_" + str(i + 1) + "/"
    print('\nIter # ', str(i+1), 'starting\n')
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_f1_score",
            patience=12,
            restore_best_weights=True,
            verbose=1,
            mode="max",
        ),
        keras.callbacks.ModelCheckpoint(
            "/app/_data/models/final/Eff5HandComplex/" + model_name,
            monitor="val_f1_score",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_f1_score",
            factor=0.9,
            patience=8,
            verbose=1,
            mode="max",
            min_delta=1e-4,
            min_lr=0.00000001,
        ),
        keras.callbacks.TensorBoard(
            log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
        ),
        keras.callbacks.experimental.BackupAndRestore(
            "/app/_data/models/final/Eff5HandComplex/backup/"
        ),
        keras.callbacks.TerminateOnNaN()
    ]

    gen_train = Generator(
        df=train,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=True,
        crop=False,
        resize=False,
    )
    gen_valid = Generator(
        df=valid,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        augment=False,
        crop=False,
        resize=False,
    )
    model = get_model()
    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=100,
        steps_per_epoch=train.shape[0] // BATCH_SIZE,
        validation_steps=valid.shape[0] // BATCH_SIZE,
        verbose=1,
        workers=15,
        callbacks=callbacks,
    )
    tf.keras.backend.clear_session()


Iter #  1 starting

Epoch 1/100
  1/624 [..............................] - ETA: 0s - loss: 0.7339 - acc: 0.1154 - recall: 0.5625 - precision: 0.2000 - f1_score: 0.1419WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
624/624 [==============================] - ETA: 0s - loss: 0.1305 - acc: 0.8244 - recall: 0.8536 - precision: 0.8971 - f1_score: 0.8152
Epoch 00001: val_f1_score improved from -inf to 0.86135, saving model to /app/_data/models/final/Eff5HandComplex/Eff5HandComplex_1.h5
624/624 [==============================] - 285s 456ms/step - loss: 0.1305 - acc: 0.8244 - recall: 0.8536 - precision: 0.8971 - f1_score: 0.8152 - val_loss: 0.0779 - val_acc: 0.8676 - val_recall: 0.9323 - val_precision: 0.9260 - val_f1_score: 0.8613
Epoch 2/100
624/624 [==============

In [30]:
def parse_all(file_path):
    img = tf.io.read_file(TRAIN_IMG_PATH + file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img


def predict_new(path, model):
    img = parse_all(path)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    return pred_to_labels(pred[0])

In [31]:
df_sub = pd.DataFrame(columns=["image", "labels"])
for img_name in os.listdir(TRAIN_IMG_PATH):
    pred = predict_new(img_name, model)

    df_sub = df_sub.append({"image": img_name, "labels": pred}, ignore_index=True)

print(df_sub.head())

                  image                   labels
0  bfc6d90f402f4c34.jpg  frog_eye_leaf_spot scab
1  9eb93fe282326266.jpg           powdery_mildew
2  f4cb3a8f41b413e4.jpg       frog_eye_leaf_spot
3  98322eab16bef2c1.jpg                     rust
4  dad5d6250cae80b7.jpg                  complex


In [32]:
df_sub = df_sub.merge(
    labels_21_20[["image", "labels"]],
    on="image",
    how="left",
    suffixes=("_pred", "_true"),
)

In [33]:
df_sub.to_csv("/app/sandbox/wrong_predictions/eff4/eff4_ns_cropped_2.csv")

In [35]:
df_sub[df_sub["labels_pred"] == ""]["labels_true"].value_counts()

complex                    22
scab                       10
rust                        7
frog_eye_leaf_spot          5
scab frog_eye_leaf_spot     1
rust frog_eye_leaf_spot     1
powdery_mildew complex      1
powdery_mildew              1
healthy                     1
Name: labels_true, dtype: int64

In [37]:
df_sub[df_sub["labels_pred"] != df_sub["labels_true"]]["labels_true"].value_counts()

scab frog_eye_leaf_spot            682
complex                            438
scab frog_eye_leaf_spot complex    200
frog_eye_leaf_spot complex         165
scab                               124
rust frog_eye_leaf_spot            118
rust complex                        91
powdery_mildew complex              87
rust                                74
frog_eye_leaf_spot                  71
healthy                             19
powdery_mildew                       7
Name: labels_true, dtype: int64